In [ ]:
import sys, os
from importlib import util

# RIFE & train_log 모듈
sys.path.append('ECCV2022-RIFE')
sys.path.append(os.path.join('ECCV2022-RIFE', 'train_log'))

def load_from_path(name, path):
    spec = util.spec_from_file_location(name, path)
    mod  = util.module_from_spec(spec)
    spec.loader.exec_module(mod)
    return mod

# RIFE_HDv3.py에서 Model 클래스 Load
rife_mod = load_from_path('RIFE_HDv3', 'ECCV2022-RIFE/train_log/RIFE_HDv3.py')
Model    = rife_mod.Model

In [ ]:
model = Model()
model.load_model('ECCV2022-RIFE/train_log', rank=-1)
model.eval()
model.device()

In [ ]:
from PIL import Image
import numpy as np
import torch, cv2

# 1) 디바이스 설정
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# 2) 원본 이미지 Load
img1 = Image.open('1.png').convert('RGB')
img2 = Image.open('30.png').convert('RGB')
w, h = img1.size

# 3) 32의 배수 Padding 계산
pad_w = ((w + 31)//32)*32
pad_h = ((h + 31)//32)*32

# 4) NumPy 변환 + Border 복사 Padding
np1 = np.array(img1)
np2 = np.array(img2)
pad_np1 = cv2.copyMakeBorder(np1, 0, pad_h-h, 0, pad_w-w, cv2.BORDER_REPLICATE)
pad_np2 = cv2.copyMakeBorder(np2, 0, pad_h-h, 0, pad_w-w, cv2.BORDER_REPLICATE)

# 5) Tensor 변환
t1 = torch.from_numpy(pad_np1).permute(2,0,1).unsqueeze(0).float().to(device)/255.
t2 = torch.from_numpy(pad_np2).permute(2,0,1).unsqueeze(0).float().to(device)/255.

# 6) 재귀 Interpolation
def recursive_interp(a, b, n):
    if n == 0:
        return []
    mid = model.inference(a, b, scale=1.0)
    if n == 1:
        return [mid]
    left_n = n//2
    right_n = n - left_n
    left = recursive_interp(a,   mid, left_n)
    right= recursive_interp(mid, b,   right_n)
    return left + [mid] + right

# 7) 순방향/역방향 중간 Frame 생성
num_inter = 28
with torch.no_grad():
    # 순방향: t1 → t2
    forward_mids  = recursive_interp(t1, t2, num_inter)
    # 역방향: t2 → t1
    backward_mids = recursive_interp(t2, t1, num_inter)

# 8) numpy list로 변환 + 원본 크기로 Crop
frames = [np1]
for t in forward_mids:
    arr = (t[0].permute(1,2,0).cpu().numpy() * 255).astype(np.uint8)
    frames.append(arr[:h, :w])
frames.append(np2)
for t in backward_mids:
    arr = (t[0].permute(1,2,0).cpu().numpy() * 255).astype(np.uint8)
    frames.append(arr[:h, :w])
frames.append(np1)

In [ ]:
# 9) 최종 생성
from PIL import Image
import numpy as np
import imageio

# 226×192로 Resize
pil_frames = [Image.fromarray(f).resize((226, 192)) for f in frames]

# 목표 Frame 수(120)로 Padding/Sampling
total   = len(pil_frames)
desired = 60 * 2
if total < desired:
    pil_frames += [pil_frames[-1]] * (desired - total)
elif total > desired:
    idxs = np.round(np.linspace(0, total-1, desired)).astype(int)
    pil_frames = [pil_frames[i] for i in idxs]

# Z-Half
#imageio.mimsave('Z-Half.gif', pil_frames, fps=30)

# Loop 설정 및 저장
imageio.mimsave(
    'Z-Full.gif',
    pil_frames,
    fps=30,
    loop=0) # 무한 반복